##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/validate_correctness"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/validate_correctness.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/validate_correctness.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/validate_correctness.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

# Validação de correção e equivalência numérica

Ao migrar o código do TensorFlow do TF1.x para o TF2, é uma boa prática garantir que o código migrado se comporte no TF2 da mesma maneira que no TF1.x.

Este guia aborda exemplos de código de migração com o shim de modelagem `tf.compat.v1.keras.utils.track_tf1_style_variables` aplicado aos métodos `tf.keras.layers.Layer`. Leia o [guia de mapeamento de modelos](./model_mapping.ipynb) para saber mais sobre os shims de modelagem do TF2.

Este guia detalha abordagens que você pode usar para:

- Validar a exatidão dos resultados obtidos nos modelos de treinamento usando o código migrado
- Validar a equivalência numérica do seu código em diferentes versões do TensorFlow

## Configuração

In [ ]:
!pip uninstall -y -q tensorflow

In [ ]:
# Install tf-nightly as the DeterministicRandomTestTool is available only in
# Tensorflow 2.8
!pip install -q tf-nightly

In [ ]:
!pip install -q tf_slim

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as v1

import numpy as np
import tf_slim as slim
import sys


from contextlib import contextmanager

In [ ]:
!git clone --depth=1 https://github.com/tensorflow/models.git
import models.research.slim.nets.inception_resnet_v2 as inception

Se você estiver colocando um pedaço não trivial de código de passo para frente no shim, você vai querer saber se ele está se comportando da mesma maneira que no TF1.x. Por exemplo, considere tentar colocar um modelo TF-Slim Inception-Resnet-v2 inteiro no shim como mostrado a seguir:

In [ ]:
# TF1 Inception resnet v2 forward pass based on slim layers
def inception_resnet_v2(inputs, num_classes, is_training):
  with slim.arg_scope(
    inception.inception_resnet_v2_arg_scope(batch_norm_scale=True)):
    return inception.inception_resnet_v2(inputs, num_classes, is_training=is_training)

In [ ]:
class InceptionResnetV2(tf.keras.layers.Layer):
  """Slim InceptionResnetV2 forward pass as a Keras layer"""

  def __init__(self, num_classes, **kwargs):
    super().__init__(**kwargs)
    self.num_classes = num_classes

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    is_training = training or False 
    
    # Slim does not accept `None` as a value for is_training,
    # Keras will still pass `None` to layers to construct functional models
    # without forcing the layer to always be in training or in inference.
    # However, `None` is generally considered to run layers in inference.
    
    with slim.arg_scope(
        inception.inception_resnet_v2_arg_scope(batch_norm_scale=True)):
      return inception.inception_resnet_v2(
          inputs, self.num_classes, is_training=is_training)


Acontece que essa camada funciona perfeitamente bem sem precisar de alterações (completa com rastreamento preciso de perdas de regularização).

No entanto, não é possível garantir que esse comportamento é preciso. Siga as etapas abaixo para verificar se ele realmente está se comportando como no TF1.x, ao ponto de observar a equivalência numérica perfeita. Essas etapas também podem ajudá-lo a triangular qual parte do passo para frente está causando alguma divergência em relação ao TF1.x (identificar se a divergência ocorre no passo para frente do modelo ou em alguma parte diferente do modelo).

## Primeiro passo: verifique se as variáveis ​​são criadas apenas uma vez

A primeira coisa que você precisa verificar é se construiu corretamente o modelo de uma forma que reutiliza variáveis ​​em cada chamada, em vez de criar e usar acidentalmente novas variáveis ​​a cada vez. Por exemplo, se o seu modelo cria uma nova camada Keras ou chama `tf.Variable` em cada passo para frente, ele provavelmente não está conseguindo capturar variáveis ​​e está criando variáveis novas a cada vez.

Abaixo estão dois escopos do gerenciador de contexto que você pode usar para detectar quando seu modelo está criando novas variáveis ​​e depurar qual parte do modelo está fazendo isso.

In [ ]:
@contextmanager
def assert_no_variable_creations():
  """Assert no variables are created in this context manager scope."""
  def invalid_variable_creator(next_creator, **kwargs):
    raise ValueError("Attempted to create a new variable instead of reusing an existing one. Args: {}".format(kwargs))

  with tf.variable_creator_scope(invalid_variable_creator):
    yield

@contextmanager
def catch_and_raise_created_variables():
  """Raise all variables created within this context manager scope (if any)."""
  created_vars = []
  def variable_catcher(next_creator, **kwargs):
    var = next_creator(**kwargs)
    created_vars.append(var)
    return var

  with tf.variable_creator_scope(variable_catcher):
    yield
  if created_vars:
    raise ValueError("Created vars:", created_vars)

O primeiro escopo (`assert_no_variable_creations()`) lançará um erro imediatamente assim que você tentar criar uma variável dentro do escopo. Isto permite inspecionar o stacktrace (e usar depuração interativa) para descobrir exatamente quais linhas de código criaram uma variável em vez de reutilizar uma existente.

O segundo escopo (`catch_and_raise_created_variables()`) irá lançar uma exceção no final do escopo se alguma variável for criada. Esta exceção incluirá a lista de todas as variáveis ​​criadas no escopo. Isso é útil para descobrir qual é o conjunto de todos os pesos que seu modelo está criando, caso você consiga identificar padrões gerais. No entanto, é menos útil para identificar as linhas exatas de código onde essas variáveis ​​foram criadas.

Use os dois escopos abaixo para verificar se a camada InceptionResnetV2 baseada em shim não cria nenhuma nova variável após a primeira chamada (e presumivelmente está reutilizando-as).

In [ ]:
model = InceptionResnetV2(1000)
height, width = 299, 299
num_classes = 1000

inputs = tf.ones( (1, height, width, 3))
# Create all weights on the first call
model(inputs)

# Verify that no new weights are created in followup calls
with assert_no_variable_creations():
  model(inputs)
with catch_and_raise_created_variables():
  model(inputs)

No exemplo abaixo, observe como esses decoradores trabalham em uma camada que cria novos pesos incorretamente a cada vez, em vez de reutilizar os existentes.

In [ ]:
class BrokenScalingLayer(tf.keras.layers.Layer):
  """Scaling layer that incorrectly creates new weights each time:"""

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    var = tf.Variable(initial_value=2.0)
    bias = tf.Variable(initial_value=2.0, name='bias')
    return inputs * var + bias

In [ ]:
model = BrokenScalingLayer()
inputs = tf.ones( (1, height, width, 3))
model(inputs)

try:
  with assert_no_variable_creations():
    model(inputs)
except ValueError as err:
  import traceback
  traceback.print_exc()


In [ ]:
model = BrokenScalingLayer()
inputs = tf.ones( (1, height, width, 3))
model(inputs)

try:
  with catch_and_raise_created_variables():
    model(inputs)
except ValueError as err:
  print(err)

Você pode corrigir a camada para garantir que ela crie os pesos apenas uma vez e os reutilize todas as outras vezes.

In [ ]:
class FixedScalingLayer(tf.keras.layers.Layer):
  """Scaling layer that incorrectly creates new weights each time:"""
  def __init__(self):
    super().__init__()
    self.var = None
    self.bias = None

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    if self.var is None:
      self.var = tf.Variable(initial_value=2.0)
      self.bias = tf.Variable(initial_value=2.0, name='bias')
    return inputs * self.var + self.bias

model = FixedScalingLayer()
inputs = tf.ones( (1, height, width, 3))
model(inputs)

with assert_no_variable_creations():
  model(inputs)
with catch_and_raise_created_variables():
  model(inputs)

### Solução de problemas

Aqui estão alguns motivos comuns pelos quais seu modelo pode acidentalmente criar novos pesos em vez de reutilizar os existentes:

1. Ele usa uma chamada `tf.Variable` explícita sem reutilizar as `tf.Variables` já criadas. Corrija isso verificando primeiro se ele não foi criado e depois reutilizando os existentes.
2. Ele cria uma camada ou modelo Keras diretamente no passo para frente todas as vezes (em vez de `tf.compat.v1.layers`). Corrija isso verificando primeiro se ele não foi criado e depois reuse os existentes.
3. Ele foi construído sobre `tf.compat.v1.layers`, mas falha ao atribuir um nome explícito a todos `compat.v1.layers` ou ao empacotar seu uso `compat.v1.layer` dentro de um nome `variable_scope`, fazendo com que os nomes das camadas geradas automaticamente aumentem a cada chamada do modelo. Corrija isso colocando um `tf.compat.v1.variable_scope` com nome dentro de seu método decorado com shim que empacota todo o uso de `tf.compat.v1.layers`.

## Segundo passo: verifique se as contagens, nomes e formatos das variáveis ​​correspondem

O segundo passo é garantir que sua camada em execução no TF2 crie o mesmo número de pesos, com os mesmos formatos, que o código correspondente cria no TF1.x.

Você pode verificá-los manualmente para ver se eles correspondem e depois também fazer verificações programaticamente em um teste de unidade, conforme mostrado abaixo.

In [ ]:
# Build the forward pass inside a TF1.x graph, and 
# get the counts, shapes, and names of the variables
graph = tf.Graph()
with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
  height, width = 299, 299
  num_classes = 1000
  inputs = tf.ones( (1, height, width, 3))

  out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=False)

  tf1_variable_names_and_shapes = {
      var.name: (var.trainable, var.shape) for var in tf.compat.v1.global_variables()}
  num_tf1_variables = len(tf.compat.v1.global_variables())

Em seguida, faça o mesmo para a camada empacotada em shim no TF2. Observe que o modelo também é chamado várias vezes antes de obter os pesos. Isto é feito para testar efetivamente a reutilização de variáveis.

In [ ]:
height, width = 299, 299
num_classes = 1000

model = InceptionResnetV2(num_classes)
# The weights will not be created until you call the model

inputs = tf.ones( (1, height, width, 3))
# Call the model multiple times before checking the weights, to verify variables
# get reused rather than accidentally creating additional variables
out, endpoints = model(inputs, training=False)
out, endpoints = model(inputs, training=False)

# Grab the name: shape mapping and the total number of variables separately,
# because in TF2 variables can be created with the same name
num_tf2_variables = len(model.variables)
tf2_variable_names_and_shapes = {
    var.name: (var.trainable, var.shape) for var in model.variables}

In [ ]:
# Verify that the variable counts, names, and shapes all match:
assert num_tf1_variables == num_tf2_variables
assert tf1_variable_names_and_shapes == tf2_variable_names_and_shapes

A camada InceptionResnetV2 baseada em shim passa neste teste. No entanto, no caso em que eles não correspondem, você pode executá-la através de um diff (texto ou outro) para descobrir onde estão as diferenças.

Isto pode fornecer uma pista sobre qual parte do modelo não está se comportando conforme o esperado. Com a execução eager, você pode usar pdb, depuração interativa e pontos de interrupção para se aprofundar nas partes do modelo que parecem suspeitas e depurar o que está errado com mais profundidade.

### Solução de problemas

- Preste muita atenção aos nomes de quaisquer variáveis ​​criadas diretamente por chamadas explícitas `tf.Variable` e camadas/modelos Keras, pois sua semântica de geração de nome de variável pode diferir ligeiramente entre os gráficos TF1.x e a funcionalidade do TF2, como execução antecipada (eager) e `tf.function`, mesmo que todo o resto esteja funcionando corretamente. Se este for o seu caso, ajuste seu teste para levar em conta qualquer semântica de nomenclatura ligeiramente diferente.

- Às vezes, você poderá descobrir que as `tf.Variable`, `tf.keras.layers.Layer` ou `tf.keras.Model` criados no passo para frente do seu loop de treinamento não aparecem na sua lista de variáveis ​​no TF2, mesmo que tenham sido capturados pela coleção de variáveis no TF1.x. Corrija esse problema atribuindo as variáveis/camadas/modelos que seu passo para frente cria a atributos de instância no seu modelo. Veja mais informações [aqui](https://www.tensorflow.org/guide/keras/custom_layers_and_models).

## Terceiro passo: redefina todas as variáveis, verifique a equivalência numérica com toda a aleatoriedade desativada

O próximo passo é verificar a equivalência numérica tanto para as saídas reais quanto para o rastreamento de perda de regularização ao corrigir o modelo de forma que não haja geração de números aleatórios envolvida (como durante a inferência).

A maneira correta de fazer isso pode depender do seu modelo específico, mas na maioria dos modelos (como este), você pode fazer o seguinte:

1. Inicializar os pesos com o mesmo valor sem aleatoriedade. Isto pode ser feito redefinindo-os para um valor fixo depois de terem sido criados.
2. Executar o modelo no modo de inferência para evitar o acionamento de camadas de dropout que podem ser fontes de aleatoriedade.

O código a seguir demonstra como você pode comparar os resultados do TF1.x e do TF2 dessa forma.

In [ ]:
graph = tf.Graph()
with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
  height, width = 299, 299
  num_classes = 1000
  inputs = tf.ones( (1, height, width, 3))

  out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=False)

  # Rather than running the global variable initializers,
  # reset all variables to a constant value
  var_reset = tf.group([var.assign(tf.ones_like(var) * 0.001) for var in tf.compat.v1.global_variables()])
  sess.run(var_reset)

  # Grab the outputs & regularization loss
  reg_losses = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
  tf1_regularization_loss = sess.run(tf.math.add_n(reg_losses))
  tf1_output = sess.run(out)

print("Regularization loss:", tf1_regularization_loss)
tf1_output[0][:5]

Obtenha os resultados no TF2.

In [ ]:
height, width = 299, 299
num_classes = 1000

model = InceptionResnetV2(num_classes)

inputs = tf.ones((1, height, width, 3))
# Call the model once to create the weights
out, endpoints = model(inputs, training=False)

# Reset all variables to the same fixed value as above, with no randomness
for var in model.variables:
  var.assign(tf.ones_like(var) * 0.001)
tf2_output, endpoints = model(inputs, training=False)

# Get the regularization loss
tf2_regularization_loss = tf.math.add_n(model.losses)

print("Regularization loss:", tf2_regularization_loss)
tf2_output[0][:5]

In [ ]:
# Create a dict of tolerance values
tol_dict={'rtol':1e-06, 'atol':1e-05}

In [ ]:
# Verify that the regularization loss and output both match
# when we fix the weights and avoid randomness by running inference:
np.testing.assert_allclose(tf1_regularization_loss, tf2_regularization_loss.numpy(), **tol_dict)
np.testing.assert_allclose(tf1_output, tf2_output.numpy(), **tol_dict)

Os números correspondem entre TF1.x e TF2 quando você remove fontes de aleatoriedade, e a camada `InceptionResnetV2` compatível com TF2 passa no teste.

Se você estiver observando resultados divergentes para seus próprios modelos, poderá usar prints ou pdb e depuração interativa para identificar onde e por que os resultados começam a divergir. A execução eager pode deixar isso bem mais fácil. Você também pode usar uma abordagem de ablação para executar apenas pequenas partes do modelo em entradas intermediárias fixas e isolar os locais onde ocorre a divergência.

Convenientemente, muitas redes slim (e outros modelos) também expõem endpoints intermediários que você pode testar.

## Quarto passo: Alinhe a geração de números aleatórios, verifique a equivalência numérica no treinamento e na inferência

O passo final é verificar se o modelo TF2 corresponde numericamente ao modelo TF1.x, mesmo quando se considera a geração de números aleatórios na inicialização da variável e no próprio passo para frente (como camadas de dropout durante o passo para frente).

Você pode fazer isso usando a ferramenta de teste abaixo para fazer com que a semântica de geração de números aleatórios corresponda entre grafos/sessões TF1.x e execução eager.

Os grafos/sessões legados do TF1 e a execução eager do TF2 usam diferentes semânticas de geração de números aleatórios stateful.

Em objetos `tf.compat.v1.Session`, se nenhuma semente for especificada, a geração de números aleatórios depende de quantas operações estão no grafo no momento em que a operação aleatória é adicionada e de quantas vezes o grafo é executado. Na execução antecipada (eager), a geração de números aleatórios stateful depende da semente global, da semente aleatória da operação e de quantas vezes a operação com a operação com a semente aleatória fornecida é executada. Veja `tf.random.set_seed` para obter mais informações.

A seguinte classe [`v1.keras.utils.DeterministicRandomTestTool`](https://www.tensorflow.org/api_docs/python/tf/compat/v1/keras/utils/DeterministicRandomTestTool) fornece um gerenciador de contexto `scope()` que pode fazer com que operações aleatórias stateful usem a mesma semente em ambos os grafos/sessões TF1 e execução antecipada (eager).

A ferramenta fornece dois modos de teste:

1. `constant` que usa a mesma semente para cada operação, não importa quantas vezes tenha sido chamada e,
2. `num_random_ops` que usa o número de operações stateful aleatórias observadas anteriormente como a semente da operação.

Isto se aplica tanto às operações aleatórias stateful usadas para criar e inicializar variáveis ​​quanto às operações aleatórias stateful usadas no cálculo (como para camadas de dropout).

Gere três tensores aleatórios para mostrar como usar esta ferramenta para fazer a correspondência de geração de números aleatórios stateful entre sessões e execução antecipada (eager).

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool()
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    a = tf.random.uniform(shape=(3,1))
    a = a * 3
    b = tf.random.uniform(shape=(3,3))
    b = b * 3
    c = tf.random.uniform(shape=(3,3))
    c = c * 3
    graph_a, graph_b, graph_c = sess.run([a, b, c])

graph_a, graph_b, graph_c

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool()
with random_tool.scope():
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  b = tf.random.uniform(shape=(3,3))
  b = b * 3
  c = tf.random.uniform(shape=(3,3))
  c = c * 3

a, b, c

In [ ]:
# Demonstrate that the generated random numbers match
np.testing.assert_allclose(graph_a, a.numpy(), **tol_dict)
np.testing.assert_allclose(graph_b, b.numpy(), **tol_dict)
np.testing.assert_allclose(graph_c, c.numpy(), **tol_dict)

Mas observe que no modo `constant`, como `b` e `c` foram gerados com a mesma semente e possuem o mesmo formato, eles terão exatamente os mesmos valores.

In [ ]:
np.testing.assert_allclose(b.numpy(), c.numpy(), **tol_dict)

### Ordem de rastreamento

Se você estiver preocupado com a correspondência de alguns números aleatórios no modo `constant`, reduzindo sua confiança em seu teste de equivalência numérica (por exemplo, se vários pesos tiverem as mesmas inicializações), você pode usar o modo `num_random_ops` para evitar isso. No modo `num_random_ops`, os números aleatórios gerados dependerão da ordem das operações aleatórias no programa.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    a = tf.random.uniform(shape=(3,1))
    a = a * 3
    b = tf.random.uniform(shape=(3,3))
    b = b * 3
    c = tf.random.uniform(shape=(3,3))
    c = c * 3
    graph_a, graph_b, graph_c = sess.run([a, b, c])

graph_a, graph_b, graph_c

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  b = tf.random.uniform(shape=(3,3))
  b = b * 3
  c = tf.random.uniform(shape=(3,3))
  c = c * 3

a, b, c

In [ ]:
# Demonstrate that the generated random numbers match
np.testing.assert_allclose(graph_a, a.numpy(), **tol_dict)
np.testing.assert_allclose(graph_b, b.numpy(), **tol_dict )
np.testing.assert_allclose(graph_c, c.numpy(), **tol_dict)

In [ ]:
# Demonstrate that with the 'num_random_ops' mode,
# b & c took on different values even though
# their generated shape was the same
assert not np.allclose(b.numpy(), c.numpy(), **tol_dict)

Entretanto, observe que neste modo a geração aleatória é sensível à ordem do programa e, portanto, os números aleatórios gerados a seguir não correspondem.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  b = tf.random.uniform(shape=(3,3))
  b = b * 3

random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  b_prime = tf.random.uniform(shape=(3,3))
  b_prime = b_prime * 3
  a_prime = tf.random.uniform(shape=(3,1))
  a_prime = a_prime * 3

assert not np.allclose(a.numpy(), a_prime.numpy())
assert not np.allclose(b.numpy(), b_prime.numpy())

Para permitir variações de depuração devido à ordem de rastreamento, a `DeterministicRandomTestTool` no modo `num_random_ops` permite ver quantas operações aleatórias foram rastreadas com a propriedade `operation_seed`.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  print(random_tool.operation_seed)
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  print(random_tool.operation_seed)
  b = tf.random.uniform(shape=(3,3))
  b = b * 3
  print(random_tool.operation_seed)

Se você precisar levar em conta a variação da ordem de rastreamento em seus testes, poderá até definir explicitamente o incremento automático com `operation_seed`. Por exemplo, você pode usar isso para fazer a correspondência de geração de números aleatórios em duas ordens de programa diferentes.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  print(random_tool.operation_seed)
  a = tf.random.uniform(shape=(3,1))
  a = a * 3
  print(random_tool.operation_seed)
  b = tf.random.uniform(shape=(3,3))
  b = b * 3

random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  random_tool.operation_seed = 1
  b_prime = tf.random.uniform(shape=(3,3))
  b_prime = b_prime * 3
  random_tool.operation_seed = 0
  a_prime = tf.random.uniform(shape=(3,1))
  a_prime = a_prime * 3

np.testing.assert_allclose(a.numpy(), a_prime.numpy(), **tol_dict)
np.testing.assert_allclose(b.numpy(), b_prime.numpy(), **tol_dict)


No entanto, a `DeterministicRandomTestTool` não permite a reutilização de sementes de operação já utilizadas, portanto, certifique-se de que as sequências incrementadas automaticamente não possam se sobrepor. Isto ocorre porque a execução antecipada (eager execution) gera números diferentes para usos subsequentes da mesma semente de operação, enquanto os grafos e sessões da TF1 não o fazem, portanto, lançar um erro ajuda a manter a sessão e a geração eager de números aleatórios stateful alinhadas.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  random_tool.operation_seed = 1
  b_prime = tf.random.uniform(shape=(3,3))
  b_prime = b_prime * 3
  random_tool.operation_seed = 0
  a_prime = tf.random.uniform(shape=(3,1))
  a_prime = a_prime * 3
  try:
    c = tf.random.uniform(shape=(3,1))
    raise RuntimeError("An exception should have been raised before this, " +
                     "because the auto-incremented operation seed will " +
                     "overlap an already-used value")
  except ValueError as err:
    print(err)


### Verificando a inferência

Agora você pode usar a `DeterministicRandomTestTool` para garantir que o modelo `InceptionResnetV2` corresponda na inferência, mesmo ao usar a inicialização de peso aleatório. Para uma condição de teste mais forte devido à ordem do programa correspondente, use o modo `num_random_ops`.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    height, width = 299, 299
    num_classes = 1000
    inputs = tf.ones( (1, height, width, 3))

    out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=False)

    # Initialize the variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Grab the outputs & regularization loss
    reg_losses = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
    tf1_regularization_loss = sess.run(tf.math.add_n(reg_losses))
    tf1_output = sess.run(out)

  print("Regularization loss:", tf1_regularization_loss)

In [ ]:
height, width = 299, 299
num_classes = 1000

random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  model = InceptionResnetV2(num_classes)

  inputs = tf.ones((1, height, width, 3))
  tf2_output, endpoints = model(inputs, training=False)

  # Grab the regularization loss as well
  tf2_regularization_loss = tf.math.add_n(model.losses)

print("Regularization loss:", tf2_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
# when using the DeterministicRandomTestTool:
np.testing.assert_allclose(tf1_regularization_loss, tf2_regularization_loss.numpy(), **tol_dict)
np.testing.assert_allclose(tf1_output, tf2_output.numpy(), **tol_dict)

### Verificando o treinamento

Já que `DeterministicRandomTestTool` funciona para *todas* as operações aleatórias stateful (incluindo inicialização e computação de pesos, como nas camadas de dropout), você também poderá usá-lo para verificar se os modelos correspondem no modo de treinamento. Você pode usar novamente o modo `num_random_ops` já que a ordem do programa das operações aleatórias stateful corresponde.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    height, width = 299, 299
    num_classes = 1000
    inputs = tf.ones( (1, height, width, 3))

    out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=True)

    # Initialize the variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Grab the outputs & regularization loss
    reg_losses = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
    tf1_regularization_loss = sess.run(tf.math.add_n(reg_losses))
    tf1_output = sess.run(out)

  print("Regularization loss:", tf1_regularization_loss)

In [ ]:
height, width = 299, 299
num_classes = 1000

random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  model = InceptionResnetV2(num_classes)

  inputs = tf.ones((1, height, width, 3))
  tf2_output, endpoints = model(inputs, training=True)

  # Grab the regularization loss as well
  tf2_regularization_loss = tf.math.add_n(model.losses)

print("Regularization loss:", tf2_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
# when using the DeterministicRandomTestTool
np.testing.assert_allclose(tf1_regularization_loss, tf2_regularization_loss.numpy(), **tol_dict)
np.testing.assert_allclose(tf1_output, tf2_output.numpy(), **tol_dict)

Agora você verificou que o modelo `InceptionResnetV2` executado de forma eager com decoradores em torno de `tf.keras.layers.Layer` corresponde numericamente à rede slim rodando em grafos e sessões TF1.

Observação: Ao usar `DeterministicRandomTestTool` no modo `num_random_ops`, é recomendado que você use e chame diretamente o decorador do método `tf.keras.layers.Layer` ao testar a equivalência numérica. Incorporá-lo num modelo funcional Keras ou em outros modelos Keras poderá produzir diferenças na ordem de rastreamento de operações aleatórias stateful que pode tornar difícil a análise ou a correspondência exata ao comparar grafos/sessões TF1.x e execução antecipada (eager).

Por exemplo, chamar a camada `InceptionResnetV2` diretamente com `training=True` intercala a inicialização da variável com a ordem de dropout de acordo com a ordem de criação da rede.

Por outro lado, primeiro colocar o decorador `tf.keras.layers.Layer` em um modelo funcional Keras e só então chamar o modelo com `training=True` equivale a inicializar todas as variáveis ​​e depois usar a camada de dropout. Isto produz uma ordem de rastreamento diferente e um conjunto diferente de números aleatórios.

No entanto, o modo padrão `mode='constant'` não é sensível a essas diferenças na ordem de rastreamento e passará sem trabalho adicional, mesmo quando a camada for incorporada num modelo funcional Keras.

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool()
with random_tool.scope():
  graph = tf.Graph()
  with graph.as_default(), tf.compat.v1.Session(graph=graph) as sess:
    height, width = 299, 299
    num_classes = 1000
    inputs = tf.ones( (1, height, width, 3))

    out, endpoints = inception_resnet_v2(inputs, num_classes, is_training=True)

    # Initialize the variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Get the outputs & regularization losses
    reg_losses = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
    tf1_regularization_loss = sess.run(tf.math.add_n(reg_losses))
    tf1_output = sess.run(out)

  print("Regularization loss:", tf1_regularization_loss)

In [ ]:
height, width = 299, 299
num_classes = 1000

random_tool = v1.keras.utils.DeterministicRandomTestTool()
with random_tool.scope():
  keras_input = tf.keras.Input(shape=(height, width, 3))
  layer = InceptionResnetV2(num_classes)
  model = tf.keras.Model(inputs=keras_input, outputs=layer(keras_input))

  inputs = tf.ones((1, height, width, 3))
  tf2_output, endpoints = model(inputs, training=True)

  # Get the regularization loss
  tf2_regularization_loss = tf.math.add_n(model.losses)

print("Regularization loss:", tf2_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
# when using the DeterministicRandomTestTool
np.testing.assert_allclose(tf1_regularization_loss, tf2_regularization_loss.numpy(), **tol_dict)
np.testing.assert_allclose(tf1_output, tf2_output.numpy(), **tol_dict)

## Passo 3b ou 4b (opcional): Teste com checkpoints pré-existentes

Depois do terceiro ou do quarto passo acima, poderá ser útil executar seus testes de equivalência numérica ao iniciar a partir de checkpoints baseados em nomes pré-existentes, se você os tiver. Isto pode testar se o carregamento do checkpoint legado está funcionando corretamente e se o modelo em si está funcionando de forma correta. O [guia Reutilizando checkpoints do TF1.x](./migrating_checkpoints.ipynb) aborda como reutilizar seus checkpoints TF1.x pré-existentes e transferi-los para checkpoints TF2.


## Testes adicionais e solução de problemas

À medida que você adiciona mais testes de equivalência numérica, você também pode optar por adicionar um teste que verifica a correspondência do cálculo do gradiente (ou até mesmo das atualizações do otimizador).

A retropropagação e a computação de gradientes são mais propensas a instabilidades numéricas de ponto flutuante do que os passos para frente do modelo. Isto significa que, à medida que seus testes de equivalência cobrirem partes menos isoladas do seu treinamento, você poderá começar a ver diferenças numéricas não triviais entre a execução completa em modo eager e seus grafos TF1. Isto pode ser causado pelas otimizações dos grafos pelo TensorFlow que fazem coisas do tipo substituir subexpressões num grafo com menos operações matemáticas.

Para saber se esse é o caso, você pode comparar seu código TF1 com uma computação TF2 acontecendo dentro de uma `tf.function` (que aplica passos de otimização de grafo como seu grafo TF1) em vez de uma computação puramente em modo eager. Alternativamente, você pode tentar usar `tf.config.optimizer.set_experimental_options` para desabilitar passos de otimização como `"arithmetic_optimization"` antes da computação do TF1 para saber se o resultado termina numericamente mais próximo dos resultados da computação no TF2. Nas suas execuções de treinamento reais, é recomendável usar `tf.function` com passos de otimização ativados por questões de desempenho, mas poderá ser útil desativá-los nos seus testes unitários de equivalência numérica.

Da mesma forma, você também poderá descobrir que os otimizadores `tf.compat.v1.train` e os otimizadores do TF2 têm propriedades numéricas de ponto flutuante ligeiramente diferentes dos otimizadores do TF2, mesmo que as fórmulas matemáticas que eles representam sejam as mesmas. É menos provável que isto seja um problema nas suas execuções de treinamento, mas pode exigir uma tolerância numérica maior em testes unitários de equivalência.